In [29]:
import json

In [30]:
with open("intents.json") as file:
	data = json.load(file)

In [31]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leinad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle
from tensorflow import keras

In [33]:
# prompt = input('enter your prompt: ')
# # issue = ['engine issue', 'battery issue', 'break problem', 'overheating']
# for intent in data["intents"]:
#     for pattern in intent["patterns"]:
#         if prompt in pattern:
#             print(intent["tag"])

In [34]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [35]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [37]:
try:
	model.load("model.tflearn")
except:
	model = tflearn.DNN(net)
	model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
	model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from c:\Users\Leinad\Documents\Tech_Villain\Leinadpython\CarFault\model.tflearn


In [38]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [39]:
def chat():
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break
		
		for intent in data["intents"]:
			for pattern in intent["patterns"]:
				if inp in pattern:
					print(intent["tag"])
		
		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = numpy.argmax(results)
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in data["intents"]:
				if tg['tag'] == tag:
					responses = tg['responses']	
			print(random.choice(responses))
			print("\n")
		else:
			print("I didnt get that, try again")

In [40]:
chat()

ENGINE MISFIRING
Check The Spark Plug: Pop up the hood and have a look at the spark plug. If it is black, clean it with a brush and test again. 
Change The Spark Plug: If the car does not start after cleaning, then you will need to change the spark plug. It is not a costly component and can be changed without much expenditure. 
Consult A Mechanic: If the misfire happens even after replacing the spark plug, then we would recommend that you get your car checked by a skilled engine mechanic as the problem could be related to the cylinder or piston head.


ENGINE MISFIRING
ENGINE MISFIRING
ENGINE MISFIRING
ENGINE MISFIRING
ENGINE MISFIRING
DEAD BATTERY
DEAD BATTERY
DEAD BATTERY
DEAD BATTERY
DEAD BATTERY
OVERHEATING ENGINE
OVERHEATING ENGINE
OVERHEATING ENGINE
OVERHEATING ENGINE
OVERHEATING ENGINE
ENGINE OIL SHORTAGE
ENGINE OIL SHORTAGE
BAD BREAK PAD
BAD BREAK PAD
LOOSE ENGINE BELT
LOOSE ENGINE BELT
BLOWN FUSE
BLOWN FUSE
FAULTY SHOCK ABSORBER
FAULTY SHOCK ABSORBER
RATTLING SUSPENSION
FLAT T

In [11]:
from sklearn.metrics import accuracy_score


In [52]:
prompt = input('enter your prompt: ')
for intent in data["intents"]:
	for pattern in intent["patterns"]:
		if prompt in pattern:
			print(intent["tag"])
		else:
			print('Please enter a vaild automobile issue')

ENGINE MISFIRING
ENGINE MISFIRING
ENGINE MISFIRING
ENGINE MISFIRING
ENGINE MISFIRING
DEAD BATTERY
DEAD BATTERY
DEAD BATTERY
DEAD BATTERY
DEAD BATTERY
OVERHEATING ENGINE
OVERHEATING ENGINE
OVERHEATING ENGINE
OVERHEATING ENGINE
OVERHEATING ENGINE
ENGINE OIL SHORTAGE
ENGINE OIL SHORTAGE
BAD BREAK PAD
BAD BREAK PAD
LOOSE ENGINE BELT
LOOSE ENGINE BELT
BLOWN FUSE
BLOWN FUSE
FAULTY SHOCK ABSORBER
FAULTY SHOCK ABSORBER
RATTLING SUSPENSION
FLAT TIRE
FUEL LEAKAGE
FUEL LEAKAGE
FUEL LEAKAGE
CLOGGED AIR FILTER
CLOGGED AIR FILTER
CLOGGED AIR FILTER


In [53]:
data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["hello", "hi", "hey"],
        },
        {
            "tag": "farewell",
            "patterns": ["bye", "goodbye", "see you"],
        },
        # Add more intents here
    ]
}


In [55]:
prompt = input('enter your prompt: ')
match_found = False

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        if prompt in pattern:
            print(intent["tag"])
            match_found = True
            break  # Exit the inner loop once a match is found

if not match_found:
    print("No matching tag found for the given prompt.")


greeting
farewell


In [51]:
prompt = input('enter your prompt: ')
match_found = False

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        if prompt in pattern:
            print(intent["tag"])
            match_found = True
            break  # Exit the inner loop once a match is found

if not match_found:
    print('Please enter a vaild automobile issue')


ENGINE MISFIRING
DEAD BATTERY
OVERHEATING ENGINE
ENGINE OIL SHORTAGE
BAD BREAK PAD
LOOSE ENGINE BELT
BLOWN FUSE
FAULTY SHOCK ABSORBER
RATTLING SUSPENSION
FLAT TIRE
FUEL LEAKAGE
CLOGGED AIR FILTER


In [59]:
prompt = input('enter your prompt: ')
found_tags = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        if prompt in pattern:
            found_tags.append(intent["tag"])
            break  # Exit the inner loop once a match is found

if found_tags:
    print("\n".join(found_tags))
else:
    print("Invalid prompt")


greeting
farewell


In [58]:
from sklearn.model_selection import train_test_split
train_data, eval_data, train_labels, eval_labels = train_test_split(training, output, test_size=0.2, random_state=42)

In [43]:
predictions = model.predict(eval_data)

# Convert the predictions to class labels (index with the highest probability)
predicted_labels = [labels[prediction.argmax()] for prediction in predictions]

# Calculate accuracy as the evaluation metric
accuracy = accuracy_score([labels[label.argmax()] for label in eval_labels], predicted_labels)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [44]:
import numpy as np

# Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
def calculate_tp_fp_fn(y_true, y_pred):
    tp = np.sum(np.logical_and(y_true == 1, y_pred == 1))
    fp = np.sum(np.logical_and(y_true == 0, y_pred == 1))
    fn = np.sum(np.logical_and(y_true == 1, y_pred == 0))
    return tp, fp, fn

# Evaluate precision and recall
def evaluate_precision_recall(y_true, y_pred):
    tp, fp, fn = calculate_tp_fp_fn(y_true, y_pred)
    precision = tp / (tp + fp + 1e-8)  # Add small epsilon to avoid division by zero
    recall = tp / (tp + fn + 1e-8)     # Add small epsilon to avoid division by zero
    return precision, recall

# Assuming you already have the predictions and ground truth labels
# predictions contains the predicted labels (converted to class indices)
# eval_labels contains the ground truth labels (converted to class indices)

# Convert predictions and eval_labels to binary format for multilabel evaluation
y_pred_binary = np.zeros_like(predictions)
y_pred_binary[np.arange(len(predictions)), predictions.argmax(axis=1)] = 1

eval_labels_binary = np.zeros_like(eval_labels)
eval_labels_binary[np.arange(len(eval_labels)), eval_labels.argmax(axis=1)] = 1

# Evaluate precision and recall
precision, recall = evaluate_precision_recall(eval_labels_binary, y_pred_binary)

print("Precision:", precision)
print("Recall:", recall)


Precision: 0.9999999975
Recall: 0.9999999975
